In [16]:
import time
import logging
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

class ImprovedPerplexityAutomator:
    def __init__(self, headless=False, max_tabs=5):
        self.headless = headless
        self.max_tabs = max_tabs
        self.driver = None
        self.wait = None
        self.setup_logging()
        
    def setup_logging(self):
        """로깅 설정"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('perplexity_automation.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def wait_for_manual_setup_improved(self, max_retries=3):
        """개선된 수동 설정 대기 함수"""
        print("\n" + "="*60)
        print("🔧 수동 설정 단계")
        print("="*60)
        print("다음 작업을 각 탭에서 수동으로 완료해주세요:")
        print("1. 로그인 (구글, 애플, 또는 이메일)")
        print("2. 원하는 AI 모델 선택 (Pro 모델 등)")
        print("3. 모든 탭에서 설정이 완료되면 아무 키나 눌러주세요")
        print("="*60)
        
        for retry in range(max_retries):
            input(f"⏳ 모든 탭 설정 완료 후 Enter를 눌러주세요... (시도 {retry + 1}/{max_retries})")
            
            all_tabs_ready = True
            failed_tabs = []
            
            for i, handle in enumerate(self.tab_handles):
                try:
                    self.driver.switch_to.window(handle)
                    time.sleep(2)  # 탭 전환 후 안정화 대기
                    
                    if self.verify_tab_ready_status(i + 1):
                        print(f"✅ 탭 {i+1} 설정 완료 확인됨")
                    else:
                        print(f"❌ 탭 {i+1} 설정이 완료되지 않았습니다")
                        failed_tabs.append(i + 1)
                        all_tabs_ready = False
                        
                except Exception as e:
                    self.logger.error(f"탭 {i+1} 확인 중 오류: {e}")
                    failed_tabs.append(i + 1)
                    all_tabs_ready = False
            
            if all_tabs_ready:
                print("✅ 모든 탭 설정 완료 확인됨")
                return True
            else:
                if retry < max_retries - 1:
                    print(f"⚠️ 실패한 탭들: {failed_tabs}")
                    print("다시 설정을 확인해주세요...")
                    time.sleep(3)
        
        print(f"❌ {max_retries}번 시도 후에도 일부 탭 설정이 완료되지 않았습니다")
        return False

    def verify_tab_ready_status(self, tab_number):
        """탭이 실제로 사용 준비가 되었는지 확인"""
        try:
            # 1. 로그인 상태 확인
            if not self.check_login_status():
                self.logger.warning(f"탭 {tab_number}: 로그인되지 않음")
                return False
            
            # 2. 질문 입력창 확인 (여러 셀렉터 시도)
            input_selectors = [
                "textarea[placeholder*='Ask']",
                "textarea[placeholder*='질문']",
                "textarea[placeholder*='무엇이든']",
                "textarea[data-testid='search-input']",
                "textarea[class*='search']",
                "div[contenteditable='true']",
                "textarea"
            ]
            
            input_element = None
            for selector in input_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and element.is_enabled():
                            # 실제로 클릭 가능한지 테스트
                            element.click()
                            time.sleep(0.5)
                            input_element = element
                            break
                    if input_element:
                        break
                except:
                    continue
            
            if not input_element:
                self.logger.warning(f"탭 {tab_number}: 질문 입력창을 찾을 수 없음")
                return False
            
            # 3. 페이지 로딩 완료 확인
            if not self.check_page_loaded():
                self.logger.warning(f"탭 {tab_number}: 페이지 로딩 미완료")
                return False
            
            # 4. Pro 모델 사용 가능 확인 (선택사항)
            self.check_pro_model_status(tab_number)
            
            return True
            
        except Exception as e:
            self.logger.error(f"탭 {tab_number} 상태 확인 중 오류: {e}")
            return False

    def check_login_status(self):
        """로그인 상태 확인"""
        try:
            # 로그인 버튼이 없으면 로그인된 상태
            login_indicators = [
                "button[class*='login']",
                "a[href*='login']",
                "button:contains('Sign in')",
                "button:contains('로그인')"
            ]
            
            for selector in login_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and 'login' in element.text.lower():
                            return False
                except:
                    continue
            
            # 사용자 프로필 요소가 있으면 로그인된 상태
            profile_indicators = [
                "[class*='profile']",
                "[class*='user']",
                "[class*='avatar']",
                "img[alt*='profile']"
            ]
            
            for selector in profile_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        return True
                except:
                    continue
            
            return True  # 기본적으로 로그인된 것으로 가정
            
        except Exception as e:
            self.logger.error(f"로그인 상태 확인 중 오류: {e}")
            return False

    def check_page_loaded(self):
        """페이지 로딩 완료 확인"""
        try:
            # JavaScript로 페이지 로딩 상태 확인
            ready_state = self.driver.execute_script("return document.readyState")
            if ready_state != "complete":
                return False
            
            # 로딩 스피너가 없는지 확인
            loading_selectors = [
                "[class*='loading']",
                "[class*='spinner']",
                "[class*='skeleton']",
                ".loading",
                ".spinner"
            ]
            
            for selector in loading_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        return False
                except:
                    continue
            
            return True
            
        except Exception as e:
            self.logger.error(f"페이지 로딩 상태 확인 중 오류: {e}")
            return False

    def check_pro_model_status(self, tab_number):
        """Pro 모델 상태 확인 (선택사항)"""
        try:
            pro_indicators = [
                "[class*='pro']",
                "[class*='premium']",
                "text*='Pro'",
                "text*='GPT-4'"
            ]
            
            for selector in pro_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        self.logger.info(f"탭 {tab_number}: Pro 모델 감지됨")
                        return True
                except:
                    continue
            
            self.logger.info(f"탭 {tab_number}: 기본 모델 사용 중")
            return False
            
        except Exception as e:
            self.logger.error(f"Pro 모델 상태 확인 중 오류: {e}")
            return False

    def find_question_input_improved(self):
        """개선된 질문 입력창 찾기"""
        selectors = [
            "textarea[placeholder*='Ask']",
            "textarea[placeholder*='질문']", 
            "textarea[placeholder*='무엇이든']",
            "textarea[data-testid='search-input']",
            "textarea[class*='search']",
            "div[contenteditable='true'][role='textbox']",
            "textarea[rows]",
            "textarea"
        ]

        for selector in selectors:
            try:
                elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if (element.is_displayed() and 
                        element.is_enabled() and 
                        element.size['height'] > 20):  # 최소 높이 확인
                        
                        # 실제 사용 가능한지 테스트
                        try:
                            element.click()
                            time.sleep(0.5)
                            return element
                        except:
                            continue
            except Exception as e:
                self.logger.debug(f"셀렉터 {selector} 시도 중 오류: {e}")
                continue
        
        return None

    def robust_element_interaction(self, element, text, max_retries=3):
        """안정적인 요소 상호작용"""
        for attempt in range(max_retries):
            try:
                # 요소가 여전히 유효한지 확인
                if not element.is_displayed() or not element.is_enabled():
                    self.logger.warning(f"요소가 비활성화됨, 재시도 {attempt + 1}")
                    time.sleep(2)
                    continue
                
                # 스크롤하여 요소를 뷰포트에 표시
                self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(1)
                
                # 클릭 및 텍스트 입력
                element.click()
                time.sleep(0.5)
                
                # 기존 텍스트 제거
                element.clear()
                time.sleep(0.5)
                
                # 텍스트 입력
                element.send_keys(text)
                time.sleep(1)
                
                # 입력 확인
                if element.get_attribute('value') or element.text:
                    return True
                    
            except Exception as e:
                self.logger.warning(f"요소 상호작용 시도 {attempt + 1} 실패: {e}")
                time.sleep(2)
        
        return False

    def enhanced_error_handling(self):
        """향상된 에러 처리"""
        try:
            # 현재 페이지 상태 진단
            current_url = self.driver.current_url
            page_title = self.driver.title
            
            self.logger.info(f"현재 URL: {current_url}")
            self.logger.info(f"페이지 제목: {page_title}")
            
            # 에러 메시지 확인
            error_selectors = [
                "[class*='error']",
                "[class*='alert']",
                "[role='alert']",
                ".error",
                ".alert"
            ]
            
            for selector in error_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed():
                            error_text = element.text
                            if error_text:
                                self.logger.error(f"페이지 에러 메시지: {error_text}")
                except:
                    continue
            
            # 스크린샷 저장
            timestamp = int(time.time())
            screenshot_path = f"error_screenshot_{timestamp}.png"
            self.driver.save_screenshot(screenshot_path)
            self.logger.info(f"에러 스크린샷 저장: {screenshot_path}")
            
        except Exception as e:
            self.logger.error(f"에러 처리 중 추가 오류: {e}")

    

    def start_driver(self):
        try:
            # ChromeDriverManager를 사용하여 자동으로 호환되는 버전 설치
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=self.chrome_options)
            self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            self.wait = WebDriverWait(self.driver, 30)
            print("✅ WebDriver 시작 완료")
            return True
        except Exception as e:
            print(f"❌ WebDriver 시작 실패: {e}")
            return False

    def stop_driver(self):
        if self.driver:
            self.driver.quit()
            print("✅ WebDriver 종료 완료")

    def open_perplexity(self):
        try:
            print("🌐 Perplexity 웹사이트 접속 중...")
            self.driver.get("https://www.perplexity.ai")
            time.sleep(3)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("✅ Perplexity 웹사이트 접속 완료")
            return True
        except Exception as e:
            print(f"❌ Perplexity 접속 실패: {e}")
            return False

    def create_multiple_tabs(self):
        """여러 탭을 생성하고 각각 Perplexity로 이동"""
        try:
            print(f"📑 {self.max_tabs}개의 탭 생성 중...")
            
            # 첫 번째 탭은 이미 열려있음
            self.tab_handles.append(self.driver.current_window_handle)
            self.tab_states[self.driver.current_window_handle] = {
                'status': 'ready',  # ready, questioning, answering, completed
                'question': None,
                'question_index': None
            }
            
            # 추가 탭들 생성
            for i in range(1, self.max_tabs):
                self.driver.execute_script("window.open('https://www.perplexity.ai', '_blank');")
                time.sleep(3)
                
                # 새로 생성된 탭으로 전환
                new_handles = self.driver.window_handles
                new_handle = [h for h in new_handles if h not in self.tab_handles][0]
                self.tab_handles.append(new_handle)
                
                # 탭 상태 초기화
                self.tab_states[new_handle] = {
                    'status': 'ready',
                    'question': None,
                    'question_index': None
                }
                
                # 새 탭으로 전환하여 페이지 로드 확인
                self.driver.switch_to.window(new_handle)
                self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                print(f"✅ 탭 {i+1} 생성 완료")
                time.sleep(2)
            
            print(f"✅ 총 {len(self.tab_handles)}개 탭 생성 완료")
            return True
            
        except Exception as e:
            print(f"❌ 탭 생성 실패: {e}")
            return False

    def wait_for_manual_setup(self):
        print("\n" + "="*60)
        print("🔧 수동 설정 단계")
        print("="*60)
        print("다음 작업을 각 탭에서 수동으로 완료해주세요:")
        print("1. 로그인 (구글, 애플, 또는 이메일)")
        print("2. 원하는 AI 모델 선택 (Pro 모델 등)")
        print("3. 모든 탭에서 설정이 완료되면 아무 키나 눌러주세요")
        print("="*60)
        input("⏳ 모든 탭 설정 완료 후 Enter를 눌러주세요...")
        
        # 각 탭이 준비되었는지 확인
        for i, handle in enumerate(self.tab_handles):
            try:
                self.driver.switch_to.window(handle)
                question_input = self.wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "textarea, input[type='text']"))
                )
                print(f"✅ 탭 {i+1} 설정 완료 확인됨")
            except:
                print(f"❌ 탭 {i+1} 설정이 완료되지 않았습니다")
                return False
        
        print("✅ 모든 탭 설정 완료 확인됨")
        return True

    def get_next_tab(self):
        """다음 탭으로 순차적으로 이동"""
        current_handle = self.tab_handles[self.current_tab_index]
        self.current_tab_index = (self.current_tab_index + 1) % len(self.tab_handles)
        return current_handle

    def switch_to_tab(self, tab_handle):
        """특정 탭으로 전환"""
        try:
            self.driver.switch_to.window(tab_handle)
            time.sleep(1)
            return True
        except Exception as e:
            print(f"❌ 탭 전환 실패: {e}")
            return False

    def go_to_perplexity_home(self):
        """퍼플렉시티 홈으로 이동"""
        try:
            print("🏠 퍼플렉시티 홈으로 이동 중...")
            self.driver.get("https://www.perplexity.ai")
            time.sleep(3)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("✅ 퍼플렉시티 홈 이동 완료")
            return True
        except Exception as e:
            print(f"❌ 퍼플렉시티 홈 이동 실패: {e}")
            return False

    def check_answer_status(self):
        """현재 탭의 답변 상태 확인"""
        try:
            # 로딩 중인지 확인
            loading_selectors = [
                "[data-testid='loading']",
                ".loading",
                "[class*='loading']",
                "[class*='spinner']",
                "[class*='generating']",
                "[class*='thinking']",
                "svg[class*='animate']",
                "[aria-label*='loading']",
                "[aria-label*='generating']"
            ]
            
            for selector in loading_selectors:
                try:
                    loading_elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in loading_elements:
                        if element.is_displayed():
                            return 'answering'  # 답변 생성 중
                except:
                    continue
            
            # 답변이 있는지 확인
            answer_content = self.get_current_answer_content()
            if answer_content and len(answer_content) > 50:
                return 'completed'  # 답변 완료
            
            # 질문 입력창이 있는지 확인
            input_element = self.find_question_input()
            if input_element and input_element.is_enabled():
                return 'ready'  # 질문 입력 준비
            
            return 'unknown'
            
        except Exception as e:
            print(f"❌ 답변 상태 확인 실패: {e}")
            return 'unknown'

    def wait_for_upload_complete(self, timeout=10):
        """파일 업로드 완료 대기"""
        try:
            print("📤 파일 업로드 완료 대기 중...")
            start_time = time.time()
            
            while time.time() - start_time < timeout:
                upload_indicators = [
                    "[class*='upload']",
                    "[class*='progress']",
                    "[aria-label*='upload']",
                    "[data-testid*='upload']"
                ]
                
                upload_in_progress = False
                for selector in upload_indicators:
                    try:
                        elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                        for element in elements:
                            if element.is_displayed() and ('progress' in element.get_attribute('class').lower() or 
                                                         'upload' in element.get_attribute('class').lower()):
                                upload_in_progress = True
                                break
                        if upload_in_progress:
                            break
                    except:
                        continue
                
                if not upload_in_progress:
                    try:
                        input_element = self.find_question_input()
                        if input_element and input_element.is_enabled():
                            print("✅ 파일 업로드 완료 확인")
                            time.sleep(2)
                            return True
                    except:
                        pass
                
                print("🔄 파일 업로드 진행 중...")
                time.sleep(3)
            
            print("⚠️ 업로드 대기 타임아웃 - 진행합니다")
            return True
            
        except Exception as e:
            print(f"❌ 업로드 대기 중 오류: {e}")
            return False

    def upload_html_file(self, html_file_path):
        """HTML 파일 업로드"""
        try:
            print(f"📤 HTML 파일 업로드 시작: {html_file_path}")
            absolute_path = os.path.abspath(html_file_path)
            
            upload_input = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="file"]'))
            )
            
            upload_input.send_keys(absolute_path)
            print(f"📁 파일 경로 전송 완료")
            
            if self.wait_for_upload_complete():
                print(f"✅ HTML 파일 업로드 완료: {absolute_path}")
                return True
            else:
                print(f"❌ HTML 파일 업로드 실패: {absolute_path}")
                return False
                
        except Exception as e:
            print(f"❌ HTML 파일 업로드 실패: {e}")
            return False

    def find_question_input(self):
        """질문 입력창 찾기"""
        selectors = [
            "textarea[placeholder*='질문']",
            "textarea[placeholder*='Ask']", 
            "textarea[placeholder*='무엇이든']",
            "textarea",
            "input[type='text']",
            "[contenteditable='true']"
        ]

        for selector in selectors:
            try:
                element = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                )
                if element.is_displayed() and element.is_enabled():
                    return element
            except:
                continue
        return None

    def ask_question_only(self, question):
        """질문만 입력하고 바로 리턴"""
        try:
            print(f"❓ 질문 입력 중: {question[:50]}...")

            input_element = self.find_question_input()
            if not input_element:
                print("❌ 질문 입력창을 찾을 수 없습니다")
                return False

            input_element.click()
            time.sleep(0.5)
            input_element.clear()
            time.sleep(0.5)
            
            input_element.send_keys(Keys.CONTROL + "a")
            time.sleep(0.2)
            input_element.send_keys(Keys.DELETE)
            time.sleep(0.5)

            for char in question:
                input_element.send_keys(char)
                time.sleep(0.01)

            time.sleep(2)
            input_element.send_keys(Keys.RETURN)
            time.sleep(1)

            print("✅ 질문 전송 완료")
            return True

        except Exception as e:
            print(f"❌ 질문 입력 실패: {e}")
            return False

    def get_current_answer_content(self):
        """현재 답변 내용 가져오기"""
        try:
            answer_selectors = [
                "[data-testid='answer']",
                "[class*='answer']",
                "[class*='response']",
                "[class*='message']",
                "main div div div",
                "article",
                ".prose"
            ]
            
            for selector in answer_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed():
                            text = element.text.strip()
                            if len(text) > 50:
                                return text
                except:
                    continue
            
            body = self.driver.find_element(By.TAG_NAME, "body")
            return body.text.strip()
        except:
            return ""

    def extract_answer(self):
        """답변 추출"""
        try:
            print("📄 답변 추출 중...")
            answer_selectors = [
                "[data-testid='answer']",
                "[class*='answer']",
                "[class*='response']",
                "[class*='message']",
                "main div div div",
                "article",
                ".prose"
            ]
            
            answer_text = ""
            answer_html = ""
            
            for selector in answer_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and element.text.strip():
                            text = element.text.strip()
                            if len(text) > 50:
                                answer_text = text
                                answer_html = element.get_attribute('outerHTML')
                                break
                    if answer_text:
                        break
                except Exception as e:
                    continue
            
            if not answer_text:
                body = self.driver.find_element(By.TAG_NAME, "body")
                answer_html = body.get_attribute('outerHTML')
                answer_text = body.text
            
            print(f"✅ 답변 추출 완료 (길이: {len(answer_text)} 문자)")
            return answer_text, answer_html
            
        except Exception as e:
            print(f"❌ 답변 추출 실패: {e}")
            return "", ""

    def extract_html_code_from_answer(self, answer_text):
        """답변에서 HTML 코드 추출"""
        html_patterns = [
            r'``````',
            r'``````',
            r'(<\!DOCTYPE.*?</html>)'
        ]
        
        for pattern in html_patterns:
            matches = re.findall(pattern, answer_text, re.DOTALL | re.IGNORECASE)
            if matches:
                return max(matches, key=len).strip()
        return ""

    def extract_filename_from_canonical(self, html_code):
        """Canonical link에서 파일명 추출"""
        try:
            soup = BeautifulSoup(html_code, 'html.parser')
            canonical_link = soup.find('link', rel='canonical')
            if canonical_link:
                href = canonical_link.get('href')
                if href:
                    filename = href.split('/')[-1]
                    print(f"📋 Canonical link에서 파일명 추출: {filename}")
                    return filename
            return None
        except Exception as e:
            print(f"❌ Canonical link 추출 실패: {e}")
            return None

    def save_html_file(self, html_code, question, index):
        """HTML 파일 저장"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            canonical_filename = self.extract_filename_from_canonical(html_code)
            
            if canonical_filename:
                filename = f"{canonical_filename}"
            else:
                question_clean = re.sub(r'[<>:"/\\|?*]', '', question)
                question_clean = re.sub(r'[^\w\s-]', '', question_clean)
                question_clean = re.sub(r'\s+', '_', question_clean.strip())[:20]
                filename = f"{timestamp}_{question_clean}.html"
            
            filepath = self.output_dir / filename
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(html_code)
            
            print(f"✅ HTML 파일 저장 완료: {filename}")
            return filepath
            
        except Exception as e:
            print(f"❌ HTML 파일 저장 실패: {e}")
            return None

    def process_tab_smartly(self, tab_handle, html_file_path):
        """탭을 스마트하게 처리 (답변 완료 확인 후 저장, 새 질문 진행)"""
        try:
            tab_number = self.tab_handles.index(tab_handle) + 1
            tab_state = self.tab_states[tab_handle]
            
            print(f"\n🔍 탭 {tab_number} 상태 확인 중...")
            
            # 해당 탭으로 전환
            if not self.switch_to_tab(tab_handle):
                return False
            
            # 현재 답변 상태 확인
            answer_status = self.check_answer_status()
            print(f"📊 탭 {tab_number} 상태: {answer_status}")
            
            # 답변이 완료된 경우 먼저 저장
            if answer_status == 'completed' and tab_state['status'] == 'answering':
                print(f"💾 탭 {tab_number}에서 완료된 답변 저장 중...")
                
                # 답변 추출
                answer_text, answer_html = self.extract_answer()
                if answer_text:
                    # HTML 코드 추출 및 저장
                    html_code = self.extract_html_code_from_answer(answer_text)
                    if html_code:
                        saved_file = self.save_html_file(
                            html_code, 
                            tab_state['question'], 
                            tab_state['question_index']
                        )
                        if saved_file:
                            print(f"✅ 탭 {tab_number} 답변 저장 완료")
                            # 상태 업데이트
                            tab_state['status'] = 'completed'
                        else:
                            print(f"❌ 탭 {tab_number} 답변 저장 실패")
                            return False
                    else:
                        print(f"❌ 탭 {tab_number}에서 HTML 코드 추출 실패")
                        return False
                else:
                    print(f"❌ 탭 {tab_number}에서 답변 추출 실패")
                    return False
                
                # 저장 완료 후 홈으로 이동
                if not self.go_to_perplexity_home():
                    return False
                
                # 상태를 ready로 변경
                tab_state['status'] = 'ready'
                tab_state['question'] = None
                tab_state['question_index'] = None
            
            # 새로운 질문이 있고 탭이 준비된 경우
            if (tab_state['status'] == 'ready' and 
                self.current_question_index < len(self.question_queue)):
                
                question = self.question_queue[self.current_question_index]
                question_index = self.current_question_index + 1
                
                print(f"🆕 탭 {tab_number}에서 새 질문 {question_index} 시작...")
                
                # HTML 파일 업로드
                if not self.upload_html_file(html_file_path):
                    print(f"❌ 탭 {tab_number}에서 HTML 파일 업로드 실패")
                    return False
                
                # 질문 입력
                if self.ask_question_only(question):
                    # 탭 상태 업데이트
                    tab_state['status'] = 'answering'
                    tab_state['question'] = question
                    tab_state['question_index'] = question_index
                    
                    # 다음 질문으로 이동
                    self.current_question_index += 1
                    
                    print(f"✅ 탭 {tab_number}에서 질문 {question_index} 제출 완료")
                    return True
                else:
                    print(f"❌ 탭 {tab_number}에서 질문 입력 실패")
                    return False
            
            return True
            
        except Exception as e:
            print(f"❌ 탭 {tab_number} 처리 중 오류: {e}")
            return False

    def run_smart_tab_automation(self, questions, html_file_path):
        """스마트 탭 자동화 실행"""
        if not questions:
            print("❌ 질문 목록이 비어있습니다")
            return

        self.question_queue = questions
        self.current_question_index = 0
        
        print(f"\n🚀 스마트 탭 자동화 시작 - 총 {len(questions)}개 질문, {self.max_tabs}개 탭 사용")
        print("="*60)
        
        # 모든 질문이 완료될 때까지 반복
        max_iterations = len(questions) * 30  # 안전장치
        iteration = 0
        
        while self.current_question_index < len(questions) or self.has_active_questions():
            iteration += 1
            if iteration > max_iterations:
                print("⚠️ 최대 반복 횟수 초과 - 종료합니다")
                break
            
            print(f"\n🔄 반복 {iteration} - 진행률: {self.current_question_index}/{len(questions)}")
            
            # 모든 탭을 순회하며 처리
            for tab_handle in self.tab_handles:
                if not self.process_tab_smartly(tab_handle, html_file_path):
                    print(f"❌ 탭 처리 실패")
                
                # 잠시 대기
                time.sleep(2)
            
            # 모든 질문이 제출되고 모든 답변이 완료되었는지 확인
            if (self.current_question_index >= len(questions) and 
                not self.has_active_questions()):
                print("✅ 모든 질문 처리 완료!")
                break
            
            # 다음 라운드 전 대기
            print("⏳ 다음 라운드 전 대기...")
            time.sleep(5)
        
        # 결과 요약
        completed_count = sum(1 for state in self.tab_states.values() 
                            if state['status'] == 'completed')
        print(f"\n📊 자동화 완료 - 완료된 질문: {completed_count}/{len(questions)}")
        
        return True

    def has_active_questions(self):
        """아직 처리 중인 질문이 있는지 확인"""
        for state in self.tab_states.values():
            if state['status'] in ['questioning', 'answering']:
                return True
        return False

def main():
    questions = [
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 카라카스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 마라카이보, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 발렌시아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 바르키시메토, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 마라카이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 시우다드과야나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 시우다드볼리바르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 바르셀로나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 산크리스토발, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 마투린, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 쿠마나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 푸에르토라크루스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 산펠리페, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 아카리과, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 바리나스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 메리다, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 투르메로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 카비마스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 푸에르토오르다스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 발레라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 카루파노, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 코로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 아라과, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 카보레데로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 엘티그레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 과나레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 과이레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 라과이라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 로스테케스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 마카포, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 푸에르토아야쿠초, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 산카를로스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 산페르난도데아푸레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 베네수엘라의 도시인 투쿠피타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 키토, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 과야킬, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 쿠엥카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 산타엘레나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 마찰라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 산토도밍고, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 포르토비에호, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 만타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 두란, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 암바토, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 리오밤바, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 라타쿤가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 로하, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 이바라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 바바오요, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 밀라그로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 다울레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 에콰도르의 도시인 라리베르타드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 라파스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 산타크루스데라시에라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 수크레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 코차밤바, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 오루로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 포토시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 타리하, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 트리니다드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 코비하, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 루레나바케, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 우유니, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 코파카바나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 베니, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 산타아나델야쿠마, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 리베랄타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 과야라메린, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 산보르하, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 산이그나시오데벨라스코, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 산호세데치키토스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 몬테로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 바야그란데, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 사마이파타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 소라타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 볼리비아의 도시인 아포로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 아순시온, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 시우다드델에스테, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 엥카르나시온, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 루케, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 카피아타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 산로렌소, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 람바레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 페르난도데라모라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 네음비, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 마리아노로케알론소, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 이타우과, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 콘셉시온, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 코로넬오비에도, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 페드로후안카바예로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 카사파, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 비야리카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 산후안바우티스타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 카쿠페, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 아레과, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파라과이의 도시인 필라르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 몬테비데오, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 살토, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 시우다드데라코스타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 파이산두, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 라스피에드라스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 리베라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 말도나도, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 멜로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 타콰렘보, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 아르티가스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 두라스노, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 플로레스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 트레인타이트레스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 로차, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 미나스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 메르세데스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 산호세데마요, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 카넬로네스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 콜로니아델사크라멘토, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 산카를로스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 산타루시아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 살리나스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 푼타델에스테, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 피리아폴리스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 우루과이의 도시인 카멜로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 조지타운, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 린든, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 뉴암스테르담, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 코리반, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 바르티카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 로즈홀, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 안나레기나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 마하이카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 마하이코니, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 파리카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 스프링랜즈, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 니켈리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 가이아나의 도시인 레테름, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 파라마리보, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 니우니케리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 니우암스테르담, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 모엔고, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 와게닝엔, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 코티카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 콰말라사무투, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 토트너스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 펠레루테푸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 포키그론, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 아나파이커, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 아베나스턴, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 아우로라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 아페티나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 아포에라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 알리안커, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 바타비아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 반하티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 벤즈도르프, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 수리남의 도시인 렐리도르프, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 카옌, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 쿠루, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 생로랑뒤마로니, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 마나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 생조르주드로야팍, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 마리파술라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 그랑상티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 아파투, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 레지나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 로우라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 아와라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 파파이숑, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 캄피, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 몽시네리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 프랑스령기아나의 도시인 사울, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 오클랜드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 웰링턴, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 크라이스트처치, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 해밀턴, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 타우랑가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 더니든, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 넬슨, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 로토루아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 팔머스턴노스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 네이피어, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 인버카길, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 왕가레이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 퀸스타운, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 기즈번, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 블레넘, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 그레이마우스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 리치먼드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 팡가레이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 파카타네, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 뉴질랜드의 도시인 스트랫퍼드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 수바, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 라우토카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 나디, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 라바사, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 시가토카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 바, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 나우소리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 코로레부, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 사부사부, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 피지의 도시인 레부카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 포트모르즈비, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 라에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 마운트하겐, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 마당, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 웨와크, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 반디아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 케렌가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 고로카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 포폰데타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 다루, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 키메나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 반자, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 알로타우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 케라베트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 멘디, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 테베레베, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 바이니모, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 반와라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 로레나우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파푸아뉴기니의 도시인 사마라이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 호니아라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 아우키, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 기조, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 툴라기, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 키라키라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 부알라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 라타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 테마투아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 타로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 솔로몬제도의 도시인 포인트크루즈, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 포트빌라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 루간빌, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 이상겔, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 라카토로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 롱가나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 바누아투의 도시인 솔라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 아피아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 사말라엘루, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 팔레올로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 루아이마투, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 살레라바이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 파가파가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 파가이로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 루피, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 파가이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 사파타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 사팔레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 살라일루아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 파가이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 팔레마, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 사모아의 도시인 파가이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 누쿠알로파, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 네이아푸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 판가이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 바바우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 하아파이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 오하, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 홀레바, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 무아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 팡가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 통가의 도시인 콜로바이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 푸나푸티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 바이투푸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 나누메아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 나누망가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 니우타오, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 누이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 누쿠펫타우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 투발루의 도시인 누쿠라엘라에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 야렌, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 아이우에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 아나바르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 아니바레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 바이티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 보에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 데니고무두, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 에와, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 이주, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 멘엥, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 니보크, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 나우루의 도시인 우아보에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 아일링나에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 아일링라팔랍, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 마로, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 아일룩, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 비카르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 론겔랍, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 마셜제도의 도시인 알링라에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 키르기스스탄의 도시인 발릭치, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 오만의 도시인 카사브, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 부탄의 도시인 푼차, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 부탄의 도시인 담추, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 부탄의 도시인 티크푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 바티칼로아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 콜론나와, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 자엘라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 에라부르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 발베티투라이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 바부니야, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 와테가마, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 스리랑카의 도시인 라트나푸라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 가지푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 나라양간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 쿨나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 랑푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 마이멘싱, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 라지샤히, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 실렛, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 코밀라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 바리살, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 보그라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 제소르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 딘아지푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 파리드푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 탕가일, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 파브나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 시라지간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 나르싱디, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 쿠시티아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 하비간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 브라흐만바리아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 찬드푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 라크샤미푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 노아칼리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 페니, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 콕스바자르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 반다르반, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 카그라차리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 랑가마티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 자말푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 셰르푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 네트라코나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 키쇼르간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 마다리푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 고팔간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 샤리아트푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 피로즈푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 자할로카티, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 바르구나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 파투아칼리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 사트키라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 마그라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 메헤르푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 차파이나와브간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 나오간, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 조이푸르하트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 팬차가르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 랄모니르하트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 쿠리그람, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 닐파마리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 가이반다, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 타쿠르가온, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 방글라데시의 도시인 수남간지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 이슬라마바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 카라치, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 라호르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 라왈핀디, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 페샤와르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 퀘타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 길기트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 무자파라바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 파이살라바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 물탄, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 굴지라나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 시알코트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 사르고다, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 바하왈푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 수쿠르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 하이데라바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 미르푸르카스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 라르카나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 나와브샤, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 자코바바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 칸푸르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 마르단, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 코하트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 아보타바드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 스와트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 치트랄, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 디라이스마일칸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 반누, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 쿠람, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 와지리스탄, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 파키스탄의 도시인 탁실라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 브루나이의 도시인 반다르스리브가완, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 브루나이의 도시인 세리아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 브루나이의 도시인 쿠알라블라잇, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 브루나이의 도시인 방아르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 비엔티안, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 루앙프라방, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 방비엥, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 팍세, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 카이손폼비한, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 타케크, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 사완나켓, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 참빠삭, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 무앙싸이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 삼느아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 아타프, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 앗따쁘, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 싸막키싸이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 버깨오, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 보케오, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 볼리캄사이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 후아판, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 캄무안, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 루앙남타, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 우돔싸이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 퐁살리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 사이냐불리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 살라완, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 세콩, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 라오스의 도시인 씨앙쿠앙, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 시소폰, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 푸르사트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 스바이리엥, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 스퉁트렝, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 크라티에, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 몬돌키리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 라타나키리, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 프레아비히어, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 오다르민체이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 캄퐁스푸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 캄퐁치낭, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 캄퐁톰, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 칸달, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 프레이벵, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 타케오, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 케프, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 팔린, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 반테아이민체이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 센모노롬, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 룽, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 스퉁트렝, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 삼라옹, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 수옹, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 타크마우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 캄퐁클랑, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 캄퐁플룩, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 종크네아스, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 아크레이크사트르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 롱벡, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 프레아닥, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 카오커, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 캄보디아의 도시인 스래암벨, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 네피도, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 양곤, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 만달레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 바간, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 인레, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 낭셰, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 몰러먀잉, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 시뛔, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 따웅지, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 하카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 롸잉꼬, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 퍼암, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 미찌나, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 뻐떼인, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 더왜, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 머궤, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 버고, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 저가잉, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 몬유아, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 떠웅구, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 삐, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 메이틸라, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 냐웅우, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 파코쿠, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 차욱, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 민부, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 예난자웅, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 타웅드윈기, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 마웅도, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 뻐떼인, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 다웨이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 머궤이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 카레이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 라이카, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 떠닝, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 꼬따웅, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미얀마의 도시인 미에익, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 평양직할시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 남포특별시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 함흥시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 청진시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 원산시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 신의주시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 단천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 개천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 개성특별시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 사리원시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 순천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 평성시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 해주시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 강계시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 안주시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 덕천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 김책시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 라선특별시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 구성시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 혜산시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 정주시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 희천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 회령시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 신포시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 송림시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 문천시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 만포시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 북한의 도시인 삼지연시, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 울란바토르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 다르한, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 에르데네트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 초이발산, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 무룬, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 올기, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 바얀홍고르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 울기, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 만달고비, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 사인샨드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 바룬우르트, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 알타이, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 톨, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 수흐바토르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 운두르칸, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 체체를렉, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 울라안곰, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 달란자드가드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 바가누르, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 나라이흐, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 주운모드, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 차간수브라가, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 몽골의 도시인 하르호린, 한국어로 웹사이트 만들어줘.  [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',

    
    ]
    
    html_file_path = "paste.txt"
    automator = PerplexitySmartTabAutomator(headless=False, max_tabs=2)
    
    try:
        if not automator.start_driver():
            return
        
        if not automator.open_perplexity():
            return
        
        if not automator.create_multiple_tabs():
            return
        
        if not automator.wait_for_manual_setup():
            return
        
        automator.run_smart_tab_automation(questions, html_file_path)
        
        print("\n🎉 모든 작업이 완료되었습니다!")
        print(f"📁 HTML 파일들은 '{automator.output_dir}' 폴더에 저장되었습니다.")
        
    except KeyboardInterrupt:
        print("\n⚠️ 사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 예상치 못한 오류 발생: {e}")
    finally:
        automator.stop_driver()

if __name__ == "__main__":
    main()


✅ WebDriver 시작 완료
🌐 Perplexity 웹사이트 접속 중...
✅ Perplexity 웹사이트 접속 완료
📑 2개의 탭 생성 중...
✅ 탭 2 생성 완료
✅ 총 2개 탭 생성 완료

🔧 수동 설정 단계
다음 작업을 각 탭에서 수동으로 완료해주세요:
1. 로그인 (구글, 애플, 또는 이메일)
2. 원하는 AI 모델 선택 (Pro 모델 등)
3. 모든 탭에서 설정이 완료되면 아무 키나 눌러주세요
⏳ 모든 탭 설정 완료 후 Enter를 눌러주세요...
❌ 탭 1 설정이 완료되지 않았습니다
✅ WebDriver 종료 완료


In [3]:
!pip install webdriver-manager

In [37]:
import os
import json
import time
import re
import logging
from datetime import datetime
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

class PerplexitySmartTabAutomator:
    def __init__(self, headless=False, max_tabs=5):
        self.headless = headless
        self.max_tabs = max_tabs
        self.driver = None
        self.wait = None
        self.output_dir = Path("perplexity_results")
        self.output_dir.mkdir(exist_ok=True)
        self.chrome_options = Options()
        self.setup_logging()
        
        if headless:
            self.chrome_options.add_argument('--headless=new')
        
        user_data_dir = Path("chrome_user_data")
        user_data_dir.mkdir(exist_ok=True)
        self.chrome_options.add_argument(f'--user-data-dir={user_data_dir.absolute()}')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        self.chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.chrome_options.add_experimental_option('useAutomationExtension', False)
        
        # 탭 관리용 변수들
        self.tab_handles = []
        self.current_tab_index = 0
        self.tab_states = {}  # 각 탭의 상태 추적
        self.question_queue = []  # 질문 대기열
        self.current_question_index = 0

    def setup_logging(self):
        """로깅 설정"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('perplexity_automation.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def start_driver(self):
        try:
            # ChromeDriverManager를 사용하여 자동으로 호환되는 버전 설치
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=self.chrome_options)
            self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            self.wait = WebDriverWait(self.driver, 30)
            print("✅ WebDriver 시작 완료")
            return True
        except Exception as e:
            print(f"❌ WebDriver 시작 실패: {e}")
            return False

    def stop_driver(self):
        if self.driver:
            self.driver.quit()
            print("✅ WebDriver 종료 완료")

    def open_perplexity(self):
        try:
            print("🌐 Perplexity 웹사이트 접속 중...")
            self.driver.get("https://www.perplexity.ai")
            time.sleep(3)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("✅ Perplexity 웹사이트 접속 완료")
            return True
        except Exception as e:
            print(f"❌ Perplexity 접속 실패: {e}")
            return False

    def create_multiple_tabs(self):
        """여러 탭을 생성하고 각각 Perplexity로 이동"""
        try:
            print(f"📑 {self.max_tabs}개의 탭 생성 중...")
            
            # 첫 번째 탭은 이미 열려있음
            self.tab_handles.append(self.driver.current_window_handle)
            self.tab_states[self.driver.current_window_handle] = {
                'status': 'ready',  # ready, questioning, answering, completed
                'question': None,
                'question_index': None
            }
            
            # 추가 탭들 생성
            for i in range(1, self.max_tabs):
                self.driver.execute_script("window.open('https://www.perplexity.ai', '_blank');")
                time.sleep(3)
                
                # 새로 생성된 탭으로 전환
                new_handles = self.driver.window_handles
                new_handle = [h for h in new_handles if h not in self.tab_handles][0]
                self.tab_handles.append(new_handle)
                
                # 탭 상태 초기화
                self.tab_states[new_handle] = {
                    'status': 'ready',
                    'question': None,
                    'question_index': None
                }
                
                # 새 탭으로 전환하여 페이지 로드 확인
                self.driver.switch_to.window(new_handle)
                self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                print(f"✅ 탭 {i+1} 생성 완료")
                time.sleep(2)
            
            print(f"✅ 총 {len(self.tab_handles)}개 탭 생성 완료")
            return True
            
        except Exception as e:
            print(f"❌ 탭 생성 실패: {e}")
            return False

    def wait_for_manual_setup_improved(self, max_retries=3):
        """개선된 수동 설정 대기 함수"""
        print("\n" + "="*60)
        print("🔧 수동 설정 단계")
        print("="*60)
        print("다음 작업을 각 탭에서 수동으로 완료해주세요:")
        print("1. 로그인 (구글, 애플, 또는 이메일)")
        print("2. 원하는 AI 모델 선택 (Pro 모델 등)")
        print("3. 모든 탭에서 설정이 완료되면 아무 키나 눌러주세요")
        print("="*60)
        
        for retry in range(max_retries):
            input(f"⏳ 모든 탭 설정 완료 후 Enter를 눌러주세요... (시도 {retry + 1}/{max_retries})")
            
            all_tabs_ready = True
            failed_tabs = []
            
            for i, handle in enumerate(self.tab_handles):
                try:
                    self.driver.switch_to.window(handle)
                    time.sleep(2)  # 탭 전환 후 안정화 대기
                    
                    if self.verify_tab_ready_status(i + 1):
                        print(f"✅ 탭 {i+1} 설정 완료 확인됨")
                    else:
                        print(f"❌ 탭 {i+1} 설정이 완료되지 않았습니다")
                        self.debug_tab_status(i + 1)
                        failed_tabs.append(i + 1)
                        all_tabs_ready = False
                        
                except Exception as e:
                    self.logger.error(f"탭 {i+1} 확인 중 오류: {e}")
                    failed_tabs.append(i + 1)
                    all_tabs_ready = False
            
            if all_tabs_ready:
                print("✅ 모든 탭 설정 완료 확인됨")
                return True
            else:
                if retry < max_retries - 1:
                    print(f"⚠️ 실패한 탭들: {failed_tabs}")
                    print("다시 설정을 확인해주세요...")
                    time.sleep(3)
        
        print(f"❌ {max_retries}번 시도 후에도 일부 탭 설정이 완료되지 않았습니다")
        return False

    def verify_tab_ready_status(self, tab_number):
        """탭이 실제로 사용 준비가 되었는지 확인"""
        try:
            # 1. 로그인 상태 확인
            if not self.check_login_status():
                self.logger.warning(f"탭 {tab_number}: 로그인되지 않음")
                return False
            
            # 2. 질문 입력창 확인 (여러 셀렉터 시도)
            input_selectors = [
                "textarea[placeholder*='Ask']",
                "textarea[placeholder*='질문']",
                "textarea[placeholder*='무엇이든']",
                "textarea[data-testid='search-input']",
                "textarea[class*='search']",
                "div[contenteditable='true']",
                "textarea"
            ]
            
            input_element = None
            for selector in input_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and element.is_enabled():
                            # 실제로 클릭 가능한지 테스트
                            element.click()
                            time.sleep(0.5)
                            input_element = element
                            break
                    if input_element:
                        break
                except:
                    continue
            
            if not input_element:
                self.logger.warning(f"탭 {tab_number}: 질문 입력창을 찾을 수 없음")
                return False
            
            # 3. 페이지 로딩 완료 확인
            if not self.check_page_loaded():
                self.logger.warning(f"탭 {tab_number}: 페이지 로딩 미완료")
                return False
            
            # 4. Pro 모델 사용 가능 확인 (선택사항)
            self.check_pro_model_status(tab_number)
            
            return True
            
        except Exception as e:
            self.logger.error(f"탭 {tab_number} 상태 확인 중 오류: {e}")
            return False

    def check_login_status(self):
        """로그인 상태 확인"""
        try:
            # 로그인 버튼이 없으면 로그인된 상태
            login_indicators = [
                "button[class*='login']",
                "a[href*='login']",
                "button:contains('Sign in')",
                "button:contains('로그인')"
            ]
            
            for selector in login_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and 'login' in element.text.lower():
                            return False
                except:
                    continue
            
            # 사용자 프로필 요소가 있으면 로그인된 상태
            profile_indicators = [
                "[class*='profile']",
                "[class*='user']",
                "[class*='avatar']",
                "img[alt*='profile']"
            ]
            
            for selector in profile_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        return True
                except:
                    continue
            
            return True  # 기본적으로 로그인된 것으로 가정
            
        except Exception as e:
            self.logger.error(f"로그인 상태 확인 중 오류: {e}")
            return False

    def check_page_loaded(self):
        """페이지 로딩 완료 확인"""
        try:
            # JavaScript로 페이지 로딩 상태 확인
            ready_state = self.driver.execute_script("return document.readyState")
            if ready_state != "complete":
                return False
            
            # 로딩 스피너가 없는지 확인
            loading_selectors = [
                "[class*='loading']",
                "[class*='spinner']",
                "[class*='skeleton']",
                ".loading",
                ".spinner"
            ]
            
            for selector in loading_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        return False
                except:
                    continue
            
            return True
            
        except Exception as e:
            self.logger.error(f"페이지 로딩 상태 확인 중 오류: {e}")
            return False

    def check_pro_model_status(self, tab_number):
        """Pro 모델 상태 확인 (선택사항)"""
        try:
            pro_indicators = [
                "[class*='pro']",
                "[class*='premium']",
                "text*='Pro'",
                "text*='GPT-4'"
            ]
            
            for selector in pro_indicators:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if any(el.is_displayed() for el in elements):
                        self.logger.info(f"탭 {tab_number}: Pro 모델 감지됨")
                        return True
                except:
                    continue
            
            self.logger.info(f"탭 {tab_number}: 기본 모델 사용 중")
            return False
            
        except Exception as e:
            self.logger.error(f"Pro 모델 상태 확인 중 오류: {e}")
            return False

    def debug_tab_status(self, tab_number):
        """탭 상태 디버깅"""
        try:
            current_url = self.driver.current_url
            page_title = self.driver.title
            
            print(f"탭 {tab_number} 디버그 정보:")
            print(f"  - URL: {current_url}")
            print(f"  - 제목: {page_title}")
            print(f"  - 페이지 상태: {self.driver.execute_script('return document.readyState')}")
            
            # 페이지의 모든 textarea 요소 확인
            textareas = self.driver.find_elements(By.TAG_NAME, "textarea")
            print(f"  - textarea 개수: {len(textareas)}")
            
            for i, textarea in enumerate(textareas):
                if textarea.is_displayed():
                    placeholder = textarea.get_attribute('placeholder')
                    print(f"    textarea {i}: {placeholder}")
                    
        except Exception as e:
            print(f"디버깅 중 오류: {e}")

    def get_next_tab(self):
        """다음 탭으로 순차적으로 이동"""
        current_handle = self.tab_handles[self.current_tab_index]
        self.current_tab_index = (self.current_tab_index + 1) % len(self.tab_handles)
        return current_handle

    def switch_to_tab(self, tab_handle):
        """특정 탭으로 전환"""
        try:
            self.driver.switch_to.window(tab_handle)
            time.sleep(1)
            return True
        except Exception as e:
            print(f"❌ 탭 전환 실패: {e}")
            return False

    def go_to_perplexity_home(self):
        """퍼플렉시티 홈으로 이동"""
        try:
            print("🏠 퍼플렉시티 홈으로 이동 중...")
            self.driver.get("https://www.perplexity.ai")
            time.sleep(3)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("✅ 퍼플렉시티 홈 이동 완료")
            return True
        except Exception as e:
            print(f"❌ 퍼플렉시티 홈 이동 실패: {e}")
            return False

    def check_answer_status(self):
        """현재 탭의 답변 상태 확인"""
        try:
            # 로딩 중인지 확인
            loading_selectors = [
                "[data-testid='loading']",
                ".loading",
                "[class*='loading']",
                "[class*='spinner']",
                "[class*='generating']",
                "[class*='thinking']",
                "svg[class*='animate']",
                "[aria-label*='loading']",
                "[aria-label*='generating']"
            ]
            
            for selector in loading_selectors:
                try:
                    loading_elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in loading_elements:
                        if element.is_displayed():
                            return 'answering'  # 답변 생성 중
                except:
                    continue
            
            # 답변이 있는지 확인
            answer_content = self.get_current_answer_content()
            if answer_content and len(answer_content) > 50:
                return 'completed'  # 답변 완료
            
            # 질문 입력창이 있는지 확인
            input_element = self.find_question_input_improved()
            if input_element and input_element.is_enabled():
                return 'ready'  # 질문 입력 준비
            
            return 'unknown'
            
        except Exception as e:
            print(f"❌ 답변 상태 확인 실패: {e}")
            return 'unknown'

    def wait_for_upload_complete(self, timeout=30):
        """파일 업로드 완료 대기"""
        try:
            print("📤 파일 업로드 완료 대기 중...")
            start_time = time.time()
            
            while time.time() - start_time < timeout:
                upload_indicators = [
                    "[class*='upload']",
                    "[class*='progress']",
                    "[aria-label*='upload']",
                    "[data-testid*='upload']"
                ]
                
                upload_in_progress = False
                for selector in upload_indicators:
                    try:
                        elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                        for element in elements:
                            if element.is_displayed() and ('progress' in element.get_attribute('class').lower() or 
                                                         'upload' in element.get_attribute('class').lower()):
                                upload_in_progress = True
                                break
                        if upload_in_progress:
                            break
                    except:
                        continue
                
                if not upload_in_progress:
                    try:
                        input_element = self.find_question_input_improved()
                        if input_element and input_element.is_enabled():
                            print("✅ 파일 업로드 완료 확인")
                            time.sleep(2)
                            return True
                    except:
                        pass
                
                print("🔄 파일 업로드 진행 중...")
                time.sleep(3)
            
            print("⚠️ 업로드 대기 타임아웃 - 진행합니다")
            return True
            
        except Exception as e:
            print(f"❌ 업로드 대기 중 오류: {e}")
            return False

    def upload_html_file(self, html_file_path):
        """HTML 파일 업로드"""
        try:
            print(f"📤 HTML 파일 업로드 시작: {html_file_path}")
            absolute_path = os.path.abspath(html_file_path)
            
            upload_input = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="file"]'))
            )
            
            upload_input.send_keys(absolute_path)
            print(f"📁 파일 경로 전송 완료")
            
            if self.wait_for_upload_complete():
                print(f"✅ HTML 파일 업로드 완료: {absolute_path}")
                return True
            else:
                print(f"❌ HTML 파일 업로드 실패: {absolute_path}")
                return False
                
        except Exception as e:
            print(f"❌ HTML 파일 업로드 실패: {e}")
            return False

    def find_question_input_improved(self):
        """개선된 질문 입력창 찾기"""
        selectors = [
            "textarea[placeholder*='Ask']",
            "textarea[placeholder*='질문']", 
            "textarea[placeholder*='무엇이든']",
            "textarea[data-testid='search-input']",
            "textarea[class*='search']",
            "div[contenteditable='true'][role='textbox']",
            "textarea[rows]",
            "textarea"
        ]

        for selector in selectors:
            try:
                elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if (element.is_displayed() and 
                        element.is_enabled() and 
                        element.size['height'] > 20):  # 최소 높이 확인
                        
                        # 실제 사용 가능한지 테스트
                        try:
                            element.click()
                            time.sleep(0.5)
                            return element
                        except:
                            continue
            except Exception as e:
                self.logger.debug(f"셀렉터 {selector} 시도 중 오류: {e}")
                continue
        
        return None

    def robust_element_interaction(self, element, text, max_retries=3):
        """안정적인 요소 상호작용"""
        for attempt in range(max_retries):
            try:
                # 요소가 여전히 유효한지 확인
                if not element.is_displayed() or not element.is_enabled():
                    self.logger.warning(f"요소가 비활성화됨, 재시도 {attempt + 1}")
                    time.sleep(2)
                    continue
                
                # 스크롤하여 요소를 뷰포트에 표시
                self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(1)
                
                # 클릭 및 텍스트 입력
                element.click()
                time.sleep(0.5)
                
                # 기존 텍스트 제거
                element.clear()
                time.sleep(0.5)
                
                # 텍스트 입력
                element.send_keys(text)
                time.sleep(1)
                
                # 입력 확인
                if element.get_attribute('value') or element.text:
                    return True
                    
            except Exception as e:
                self.logger.warning(f"요소 상호작용 시도 {attempt + 1} 실패: {e}")
                time.sleep(2)
        
        return False

    def ask_question_only(self, question):
        """질문만 입력하고 바로 리턴"""
        try:
            print(f"❓ 질문 입력 중: {question[:50]}...")

            input_element = self.find_question_input_improved()
            if not input_element:
                print("❌ 질문 입력창을 찾을 수 없습니다")
                return False

            # 안정적인 요소 상호작용 사용
            if not self.robust_element_interaction(input_element, question):
                print("❌ 질문 입력 실패")
                return False

            time.sleep(10)
            input_element.send_keys(Keys.RETURN)
            time.sleep(1)

            print("✅ 질문 전송 완료")
            return True

        except Exception as e:
            print(f"❌ 질문 입력 실패: {e}")
            return False

    def get_current_answer_content(self):
        """현재 답변 내용 가져오기"""
        try:
            answer_selectors = [
                "[data-testid='answer']",
                "[class*='answer']",
                "[class*='response']",
                "[class*='message']",
                "main div div div",
                "article",
                ".prose"
            ]
            
            for selector in answer_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed():
                            text = element.text.strip()
                            if len(text) > 50:
                                return text
                except:
                    continue
            
            body = self.driver.find_element(By.TAG_NAME, "body")
            return body.text.strip()
        except:
            return ""

    def extract_answer(self):
        """답변 추출"""
        try:
            print("📄 답변 추출 중...")
            answer_selectors = [
                "[data-testid='answer']",
                "[class*='answer']",
                "[class*='response']",
                "[class*='message']",
                "main div div div",
                "article",
                ".prose"
            ]
            
            answer_text = ""
            answer_html = ""
            
            for selector in answer_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        if element.is_displayed() and element.text.strip():
                            text = element.text.strip()
                            if len(text) > 50:
                                answer_text = text
                                answer_html = element.get_attribute('outerHTML')
                                break
                    if answer_text:
                        break
                except Exception as e:
                    continue
            
            if not answer_text:
                body = self.driver.find_element(By.TAG_NAME, "body")
                answer_html = body.get_attribute('outerHTML')
                answer_text = body.text
            
            print(f"✅ 답변 추출 완료 (길이: {len(answer_text)} 문자)")
            return answer_text, answer_html
            
        except Exception as e:
            print(f"❌ 답변 추출 실패: {e}")
            return "", ""

    def extract_html_code_from_answer(self, answer_text):
        """답변에서 HTML 코드 추출"""
        html_patterns = [
            r'``````',
            r'``````',
            r'(<\!DOCTYPE.*?</html>)'
        ]
        
        for pattern in html_patterns:
            matches = re.findall(pattern, answer_text, re.DOTALL | re.IGNORECASE)
            if matches:
                return max(matches, key=len).strip()
        return ""

    def extract_filename_from_canonical(self, html_code):
        """Canonical link에서 파일명 추출"""
        try:
            soup = BeautifulSoup(html_code, 'html.parser')
            canonical_link = soup.find('link', rel='canonical')
            if canonical_link:
                href = canonical_link.get('href')
                if href:
                    filename = href.split('/')[-1]
                    print(f"📋 Canonical link에서 파일명 추출: {filename}")
                    return filename
            return None
        except Exception as e:
            print(f"❌ Canonical link 추출 실패: {e}")
            return None

    def save_html_file(self, html_code, question, index):
        """HTML 파일 저장"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            canonical_filename = self.extract_filename_from_canonical(html_code)
            
            if canonical_filename:
                filename = f"{canonical_filename}"
            else:
                question_clean = re.sub(r'[<>:"/\\|?*]', '', question)
                question_clean = re.sub(r'[^\w\s-]', '', question_clean)
                question_clean = re.sub(r'\s+', '_', question_clean.strip())[:20]
                filename = f"{timestamp}_{question_clean}.html"
            
            filepath = self.output_dir / filename
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(html_code)
            
            print(f"✅ HTML 파일 저장 완료: {filename}")
            return filepath
            
        except Exception as e:
            print(f"❌ HTML 파일 저장 실패: {e}")
            return None

    def process_tab_smartly(self, tab_handle, html_file_path):
        """탭을 스마트하게 처리 (답변 완료 확인 후 저장, 새 질문 진행)"""
        try:
            tab_number = self.tab_handles.index(tab_handle) + 1
            tab_state = self.tab_states[tab_handle]
            
            print(f"\n🔍 탭 {tab_number} 상태 확인 중...")
            
            # 해당 탭으로 전환
            if not self.switch_to_tab(tab_handle):
                return False
            
            # 현재 답변 상태 확인
            answer_status = self.check_answer_status()
            print(f"📊 탭 {tab_number} 상태: {answer_status}")
            
            # 답변이 완료된 경우 먼저 저장
            if answer_status == 'completed' and tab_state['status'] == 'answering':
                print(f"💾 탭 {tab_number}에서 완료된 답변 저장 중...")
                
                # 답변 추출
                answer_text, answer_html = self.extract_answer()
                if answer_text:
                    # HTML 코드 추출 및 저장
                    html_code = self.extract_html_code_from_answer(answer_text)
                    if html_code:
                        saved_file = self.save_html_file(
                            html_code, 
                            tab_state['question'], 
                            tab_state['question_index']
                        )
                        if saved_file:
                            print(f"✅ 탭 {tab_number} 답변 저장 완료")
                            # 상태 업데이트
                            tab_state['status'] = 'completed'
                        else:
                            print(f"❌ 탭 {tab_number} 답변 저장 실패")
                            return False
                    else:
                        print(f"❌ 탭 {tab_number}에서 HTML 코드 추출 실패")
                        return False
                else:
                    print(f"❌ 탭 {tab_number}에서 답변 추출 실패")
                    return False
                
                # 저장 완료 후 홈으로 이동
                if not self.go_to_perplexity_home():
                    return False
                
                # 상태를 ready로 변경
                tab_state['status'] = 'ready'
                tab_state['question'] = None
                tab_state['question_index'] = None
            
            # 새로운 질문이 있고 탭이 준비된 경우
            if (tab_state['status'] == 'ready' and 
                self.current_question_index < len(self.question_queue)):
                
                question = self.question_queue[self.current_question_index]
                question_index = self.current_question_index + 1
                
                print(f"🆕 탭 {tab_number}에서 새 질문 {question_index} 시작...")
                
                # HTML 파일 업로드
                if not self.upload_html_file(html_file_path):
                    print(f"❌ 탭 {tab_number}에서 HTML 파일 업로드 실패")
                    return False
                
                # 질문 입력
                if self.ask_question_only(question):
                    # 탭 상태 업데이트
                    tab_state['status'] = 'answering'
                    tab_state['question'] = question
                    tab_state['question_index'] = question_index
                    
                    # 다음 질문으로 이동
                    self.current_question_index += 1
                    
                    print(f"✅ 탭 {tab_number}에서 질문 {question_index} 제출 완료")
                    return True
                else:
                    print(f"❌ 탭 {tab_number}에서 질문 입력 실패")
                    return False
            
            return True
            
        except Exception as e:
            print(f"❌ 탭 {tab_number} 처리 중 오류: {e}")
            return False

    def run_smart_tab_automation(self, questions, html_file_path):
        """스마트 탭 자동화 실행"""
        if not questions:
            print("❌ 질문 목록이 비어있습니다")
            return

        self.question_queue = questions
        self.current_question_index = 0
        
        print(f"\n🚀 스마트 탭 자동화 시작 - 총 {len(questions)}개 질문, {self.max_tabs}개 탭 사용")
        print("="*60)
        
        # 모든 질문이 완료될 때까지 반복
        max_iterations = len(questions) * 30  # 안전장치
        iteration = 0
        
        while self.current_question_index < len(questions) or self.has_active_questions():
            iteration += 1
            if iteration > max_iterations:
                print("⚠️ 최대 반복 횟수 초과 - 종료합니다")
                break
            
            print(f"\n🔄 반복 {iteration} - 진행률: {self.current_question_index}/{len(questions)}")
            
            # 모든 탭을 순회하며 처리
            for tab_handle in self.tab_handles:
                if not self.process_tab_smartly(tab_handle, html_file_path):
                    print(f"❌ 탭 처리 실패")
                
                # 잠시 대기
                time.sleep(2)
            
            # 모든 질문이 제출되고 모든 답변이 완료되었는지 확인
            if (self.current_question_index >= len(questions) and 
                not self.has_active_questions()):
                print("✅ 모든 질문 처리 완료!")
                break
            
            # 다음 라운드 전 대기
            print("⏳ 다음 라운드 전 대기...")
            time.sleep(5)
        
        # 결과 요약
        completed_count = sum(1 for state in self.tab_states.values() 
                            if state['status'] == 'completed')
        print(f"\n📊 자동화 완료 - 완료된 질문: {completed_count}/{len(questions)}")
        
        return True

    def has_active_questions(self):
        """아직 처리 중인 질문이 있는지 확인"""
        for state in self.tab_states.values():
            if state['status'] in ['questioning', 'answering']:
                return True
        return False

def main():
    questions = [
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MSFT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AAPL, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AMZN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GOOG, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GOOGL, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 META, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AVGO, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TSM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BRK-A, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BRK-B, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TSLA, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 JPM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 WMT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 LLY, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 V, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ORCL, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 NFLX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MA, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 XOM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 COST, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PG, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 JNJ, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 HD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BAC, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SAP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ABBV, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PLTR, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ASML, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 KO, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 UNH, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CSCO, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 IBM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TMUS, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 WFC, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CRM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BABA, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CVX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ABT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 NVO, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AMD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 NVS, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MS, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AXP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 DIS, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 LIN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 INTU, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GS, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AZN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 NOW, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 HSBC, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MCD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 T, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SHEL, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ACN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MRK, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 HDB, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 UBER, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 RTX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ISRG, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TXN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BKNG, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 RY, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CAT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 VZ, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PEP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 QCOM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ARM, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SCHW, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ADBE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BLK, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SPGI, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 C, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BSX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MUFG, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BA, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SONY, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PGR, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SPOT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TMO, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SYK, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 UL, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AMGN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 HON, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SHOP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 AMAT, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PDD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GEV, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 NEE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 DHR, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ETN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MU, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GILD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PFE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TJX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 DE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 UNP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TTE, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 PANW, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 COF, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CMCSA, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MELI, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 TD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ANET, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 CRWD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 ADP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 LRCX, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 LOW, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 SAN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BHP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BUD, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 IBN, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 APH, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 KKR, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 APP, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
        '내가 보내준 HTML 양식 그대로 미국 주식의 티커인 KLAC, 한국어로 웹사이트 만들어줘. [1],[2],[3]... 이런 인용구호는 절대 넣지마. 그대로 붙여넣을거니까 전체 코드로 알려줘.',
            ]
    
    html_file_path = "paste.txt"  # 업로드할 HTML 파일 경로
    
    automator = PerplexitySmartTabAutomator(headless=False, max_tabs=1)
    
    try:
        if not automator.start_driver():
            return
        
        if not automator.open_perplexity():
            return
        
        if not automator.create_multiple_tabs():
            return
        
        if not automator.wait_for_manual_setup_improved():
            return
        
        automator.run_smart_tab_automation(questions, html_file_path)
        
    except KeyboardInterrupt:
        print("\n⚠️ 사용자에 의해 중단되었습니다")
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생: {e}")
    finally:
        automator.stop_driver()

if __name__ == "__main__":
    main()


✅ WebDriver 시작 완료
🌐 Perplexity 웹사이트 접속 중...
✅ Perplexity 웹사이트 접속 완료
📑 1개의 탭 생성 중...
✅ 총 1개 탭 생성 완료

🔧 수동 설정 단계
다음 작업을 각 탭에서 수동으로 완료해주세요:
1. 로그인 (구글, 애플, 또는 이메일)
2. 원하는 AI 모델 선택 (Pro 모델 등)
3. 모든 탭에서 설정이 완료되면 아무 키나 눌러주세요
⏳ 모든 탭 설정 완료 후 Enter를 눌러주세요... (시도 1/3)
✅ 탭 1 설정 완료 확인됨
✅ 모든 탭 설정 완료 확인됨

🚀 스마트 탭 자동화 시작 - 총 119개 질문, 1개 탭 사용

🔄 반복 1 - 진행률: 0/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: ready
🆕 탭 1에서 새 질문 1 시작...
📤 HTML 파일 업로드 시작: paste.txt
📁 파일 경로 전송 완료
📤 파일 업로드 완료 대기 중...
✅ 파일 업로드 완료 확인
✅ HTML 파일 업로드 완료: C:\Users\USER\Downloads\paste.txt
❓ 질문 입력 중: 내가 보내준 HTML 양식 그대로 미국 주식의 티커인 MSFT, 한국어로 웹사이트 만들어줘...
✅ 질문 전송 완료
✅ 탭 1에서 질문 1 제출 완료
⏳ 다음 라운드 전 대기...

🔄 반복 2 - 진행률: 1/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 3 - 진행률: 1/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 4 - 진행률: 1/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 5 - 진행률: 1/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 6 - 진행률: 1/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: a

✅ 퍼플렉시티 홈 이동 완료
🆕 탭 1에서 새 질문 5 시작...
📤 HTML 파일 업로드 시작: paste.txt
📁 파일 경로 전송 완료
📤 파일 업로드 완료 대기 중...
✅ 파일 업로드 완료 확인
✅ HTML 파일 업로드 완료: C:\Users\USER\Downloads\paste.txt
❓ 질문 입력 중: 내가 보내준 HTML 양식 그대로 미국 주식의 티커인 GOOGL, 한국어로 웹사이트 만들어...
✅ 질문 전송 완료
✅ 탭 1에서 질문 5 제출 완료
⏳ 다음 라운드 전 대기...

🔄 반복 76 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 77 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 78 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 79 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 80 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 81 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 82 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 83 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 84 - 진행률: 5/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 85 - 진행률: 5/119

🔍 탭 1 

✅ 파일 업로드 완료 확인
✅ HTML 파일 업로드 완료: C:\Users\USER\Downloads\paste.txt
❓ 질문 입력 중: 내가 보내준 HTML 양식 그대로 미국 주식의 티커인 BRK-A, 한국어로 웹사이트 만들어...
✅ 질문 전송 완료
✅ 탭 1에서 질문 9 제출 완료
⏳ 다음 라운드 전 대기...

🔄 반복 156 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 157 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 158 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 159 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 160 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 161 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 162 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 163 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 164 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 165 - 진행률: 9/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 166 - 진행률: 9/119

🔍 탭 1 상태 확인 중...


✅ 퍼플렉시티 홈 이동 완료
🆕 탭 1에서 새 질문 13 시작...
📤 HTML 파일 업로드 시작: paste.txt
📁 파일 경로 전송 완료
📤 파일 업로드 완료 대기 중...
✅ 파일 업로드 완료 확인
✅ HTML 파일 업로드 완료: C:\Users\USER\Downloads\paste.txt
❓ 질문 입력 중: 내가 보내준 HTML 양식 그대로 미국 주식의 티커인 WMT, 한국어로 웹사이트 만들어줘....
✅ 질문 전송 완료
✅ 탭 1에서 질문 13 제출 완료
⏳ 다음 라운드 전 대기...

🔄 반복 237 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 238 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 239 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 240 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 241 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 242 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 243 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 244 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 245 - 진행률: 13/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 246

📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 313 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 314 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 315 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 316 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 317 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 318 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 319 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 320 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 321 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 322 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 323 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 324 - 진행률: 17/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: answering
⏳ 다음 라운드 전 대기...

🔄 반복 325 - 진행률

...

⏳ 다음 라운드 전 대기...

🔄 반복 2541 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2542 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2543 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2544 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2545 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2546 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이

⏳ 다음 라운드 전 대기...

🔄 반복 2587 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2588 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2589 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2590 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2591 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2592 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이

⏳ 다음 라운드 전 대기...

🔄 반복 2633 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2634 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2635 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2636 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2637 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2638 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이

⏳ 다음 라운드 전 대기...

🔄 반복 2679 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2680 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2681 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2682 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2683 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2684 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이

⏳ 다음 라운드 전 대기...

🔄 반복 2725 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2726 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2727 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2728 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2729 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이: 25014 문자)
❌ 탭 1에서 HTML 코드 추출 실패
❌ 탭 처리 실패
⏳ 다음 라운드 전 대기...

🔄 반복 2730 - 진행률: 73/119

🔍 탭 1 상태 확인 중...
📊 탭 1 상태: completed
💾 탭 1에서 완료된 답변 저장 중...
📄 답변 추출 중...
✅ 답변 추출 완료 (길이